# Il teorema di Ramsey

Nel capitolo sulle applicazioni della logica proposizionale dell'handbook viene trattato come esempio il teorema di Ramsey riportandone una versione meno generale così enunciata.

> Per ogni $s,t \in \mathbb{N}$ esiste un $n \in \mathbb{N}$ tale che qualsiasi grafo con $n$ vertici ha un sotto-grafo completamente connesso di dimensione $s$ o un sotto-grafo completamente disconnesso di dimensione $t$. Inoltre se il "numero di Ramsey" $R(s,t)$ denota quel minimo $n$ per una data coppia $s$ e $t$ abbiamo: $$R(s,t) \leq R(s-1,t) + R(s,t-1)$$

Vi si nota anche, come esempio, che un semplice risultato tipo quello di Ramsey è che in ogni festa di sei persone, ci deve essere un gruppo di tre persone *ciascuna* delle quali si conosce, o un gruppo di tre persone *nessuna* delle quali si conosce e che è abitudine pensare a questo tipo di problemi in termini di un *grafo*, cioè una collezione $V$ di *vertici* con certe coppie collegate da *spigoli* presi da un insieme $E$.

Obiettivo è quello di mostrare come sia possibile riportare problemi di questo genere a problemi nell'ambito della logica proposizionale e precisamente come formulare un'enunciato proposizionale che è una tautologia se $R(s,t) \leq n$. 

L'approccio è descritto nel modo seguente:

> Indicizziamo i vertici usando interi da 1 a $n$, calcoliamo tutti i sottoinsiemi di $s$-elementi e $t$-elementi, e quindi per ciascun sottoinsieme di questi $s$ o $t$-elementi a turno, tutti i possibili sottoinsiemi di 2-elementi di questi. Vogliamo esprimere il fatto che per uno di questi insiemi di $s$-elementi, ciascuna coppia di elementi è connessa, o per uno degli insiemi di $t$-elementi, ciascuna coppia di elementi è disconnessa. La definizione locale `e[m;n]` produce una formula atomica `p_m_n` che interpretiamo come "$m$ è connesso a $n$" (o "$m$ conosce $n$", ecc.).

Vediamo cosa significa. 

In [ ]:
// Nota: il namespace HOL.AutomatedReasoning è disponibile solo da codice 
// sorgente per ora, non da nuget.

#load "../../HolFormatters.fsx"

open HOL
open HOL.AutomatedReasoning

> Indicizziamo i vertici usando interi da 1 a $n$

In [2]:
let s,t,n = 3,3,4
let vertices = [1..n]
printfn "%A" vertices

[1; 2; 3; 4]


> calcoliamo tutti i sottoinsiemi di $s$-elementi e $t$-elementi

In [3]:
printfn "%A" (allsets s vertices)

[[1; 2; 3]; [1; 2; 4]; [1; 3; 4]; [2; 3; 4]]


> e quindi per ciascun sottoinsieme di questi $s$ o $t$-elementi a turno, tutti i possibili sottoinsiemi di 2-elementi di questi

In [4]:
let yesgrps = List.map (allsets 2) (allsets s vertices)
let nogrps = List.map (allsets 2) (allsets t vertices)

printfn "%A" yesgrps

[[[1; 2]; [1; 3]; [2; 3]]; [[1; 2]; [1; 4]; [2; 4]]; [[1; 3]; [1; 4]; [3; 4]];
 [[2; 3]; [2; 4]; [3; 4]]]


Questi rappresentano i possibili gruppi di vertici connessi (o non connessi):

![Link Name](vertici.png)  

> Vogliamo esprimere il fatto che per uno di questi insiemi di $s$-elementi, ciascuna coppia di elementi è connessa, o per uno degli insiemi di $t$-elementi, ciascuna coppia di elementi è disconnessa. La definizione locale `e[m;n]` produce una formula atomica `p_m_n` che interpretiamo come "$m$ è connesso a $n$" (o "$m$ conosce $n$", ecc.).

In [6]:
let e [m;n] = "p_" + (string m) + "_" + (string n) + ":bool" |> parse_term

mk_disj (
    list_disj (List.map (list_conj << List.map e) yesgrps),
    list_disj (List.map (list_conj << List.map (fun p -> mk_not (e p))) nogrps))

|> print_term
|> fun x -> x.Replace("\\/", "\\/\n")

(p_1_2 /\ p_1_3 /\ p_2_3 \/
 p_1_2 /\ p_1_4 /\ p_2_4 \/
 p_1_3 /\ p_1_4 /\ p_3_4 \/
 p_2_3 /\ p_2_4 /\ p_3_4) \/
 ~ p_1_2 /\ ~ p_1_3 /\ ~ p_2_3 \/
 ~ p_1_2 /\ ~ p_1_4 /\ ~ p_2_4 \/
 ~ p_1_3 /\ ~ p_1_4 /\ ~ p_3_4 \/
 ~ p_2_3 /\ ~ p_2_4 /\ ~ p_3_4

Possiamo racchiudere tutto in un'unica funzione:

In [8]:
let ramsey s t n =
    let vertices = [1..n]
    let yesgrps = List.map (allsets 2) (allsets s vertices)
    let nogrps = List.map (allsets 2) (allsets t vertices)
    let e [m;n] = "p_" + (string m) + "_" + (string n) + ":bool" |> parse_term
    mk_disj (
        list_disj (List.map (list_conj << List.map e) yesgrps),
        list_disj (List.map (list_conj << List.map (fun p -> mk_not (e p))) nogrps))

e confermare che il numero 6 nell'esempio iniziale della festa è il migliore possibile cioè che $R(3,3) = 6$:

In [10]:
tautology(ramsey 3 3 4)

False

In [11]:
tautology(ramsey 3 3 5)

False

In [12]:
tautology(ramsey 3 3 6)

True